# 検索クエリーのサンプル

Azure Cognitive Search の様々な検索機能を試すことができます。
1. フルテキスト検索
1. ベクトル検索
1. ハイブリッド検索（フルテキスト＋ベクトル）
1. セマンティックハイブリッド検索（ハイブリッド＋セマンティックランカー）


In [ ]:
#!pip install azure-search-documents --pre

In [ ]:
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import Vector  
from azure.core.credentials import AzureKeyCredential

In [ ]:
openai.api_type = "azure"  
openai.api_key = "<YOUR_AZURE_OPENAI_KEY"
openai.api_base = "https://<YOUR_AZURE_OPENAI_SERVICE>.openai.azure.com/"
openai.api_version = "2023-05-15"  

service_endpoint = "https://<YOUR_AZURE_COGNITIVE_SEARCH_SERVICE>.search.windows.net"
index_name = "<YOUR_AZURE_COGNITIVE_SEARCH_INDEX>"
key = "<YOUR_AZURE_COGNITIVE_SEARCH_APIKEY>"
credential = AzureKeyCredential(key)

# Full Text Search

In [ ]:
query = "三浦義澄ってなにした人"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)

results = search_client.search(  
    search_text=query,
    select=["sourcepage", "content"],
    top=3
)  
  
for result in results:  
    print(f"Sourcepage: {result['sourcepage']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  

# Vector Search

In [ ]:
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings

In [ ]:
query = "源実友のお歌にはどのような特徴があったのでしょうか？"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=["sourcepage", "content"]
)  
  
for result in results:  
    print(f"Sourcepage: {result['sourcepage']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  

# Hybrid Search

In [ ]:
query = "１３人の合議制に含まれるメンバー一覧"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    select=["sourcepage", "content"],
    top=3
)  

for result in results:  
    print(f"Sourcepage: {result['sourcepage']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  

# Semantic Hybrid Search(Hybrid + Semantic Ranker)

In [ ]:
query = "１３人の合議制に含まれるメンバー一覧"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    select=["sourcepage", "content"],
    query_type="semantic", query_language="ja-jp", semantic_configuration_name='config1', query_caption="extractive", query_answer="extractive",
    top=3
)  

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:  
    print(f"Sourcepage: {result['sourcepage']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
   
    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")